In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,27207
2,Huelva,Confirmados PDIA 14 días,1412
3,Huelva,Tasa PDIA 14 días,"275,15248358243855"
4,Huelva,Confirmados PDIA 7 días,582
5,Huelva,Total Confirmados,27380
6,Huelva,Curados,22825
7,Huelva,Fallecidos,358


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  27207.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7074.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1169 personas en los últimos 7 días 

Un positivo PCR cada 455 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,27207.0,582.0,1412.0,513170.0,113.412709,275.152484,47.0
Huelva-Costa,16095.0,245.0,594.0,289548.0,84.614641,205.147333,37.0
Huelva (capital),7074.0,123.0,316.0,143837.0,85.513463,219.693125,26.0
Condado-Campiña,8276.0,291.0,706.0,156231.0,186.262650,451.894950,23.0
Almonte,1304.0,125.0,256.0,24507.0,510.058351,1044.599502,9.0
Cartaya,1196.0,84.0,172.0,20083.0,418.264204,856.445750,7.0
Rociana del Condado,457.0,32.0,73.0,7939.0,403.073435,919.511273,6.0
Sierra de Huelva-Andévalo Central,2423.0,38.0,84.0,67391.0,56.387351,124.645724,4.0
Ayamonte,1261.0,5.0,11.0,21104.0,23.692191,52.122820,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Almonte,1304.0,125.0,256.0,24507.0,510.058351,1044.599502,9.0
San Bartolomé de la Torre,275.0,8.0,35.0,3761.0,212.709386,930.603563,0.0
Rociana del Condado,457.0,32.0,73.0,7939.0,403.073435,919.511273,6.0
Cartaya,1196.0,84.0,172.0,20083.0,418.264204,856.445750,7.0
Lucena del Puerto,185.0,4.0,22.0,3261.0,122.661760,674.639681,0.0
Palos de la Frontera,550.0,28.0,56.0,11742.0,238.460228,476.920456,2.0
Moguer,1000.0,38.0,99.0,21867.0,173.777839,452.737001,1.0
Condado-Campiña,8276.0,291.0,706.0,156231.0,186.262650,451.894950,23.0
Campillo (El),70.0,7.0,9.0,2024.0,345.849802,444.664032,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Alosno,161.0,1.0,11.0,3933.0,25.425884,279.684719,0.0,0.090909
San Juan del Puerto,409.0,3.0,25.0,9411.0,31.877590,265.646584,1.0,0.120000
Lucena del Puerto,185.0,4.0,22.0,3261.0,122.661760,674.639681,0.0,0.181818
Bonares,224.0,5.0,26.0,6060.0,82.508251,429.042904,1.0,0.192308
Gibraleón,557.0,2.0,9.0,12737.0,15.702285,70.660281,1.0,0.222222
San Bartolomé de la Torre,275.0,8.0,35.0,3761.0,212.709386,930.603563,0.0,0.228571
Villarrasa,141.0,2.0,8.0,2211.0,90.456807,361.827227,0.0,0.250000
Aljaraque,973.0,11.0,37.0,21474.0,51.224737,172.301388,2.0,0.297297
Aracena,494.0,10.0,33.0,8255.0,121.138704,399.757723,0.0,0.303030
